In [1]:
import requests
import json
import random
import time
import pickle


model = "dRIT"

template = {
  "Answer": "",
  "Score Lexical Semantics": "",
  "Score Predicate-Argument Structure": "",
  "Score Logic": "",
  "Score Knowledge & Common Sense": ""
}


# DEV de SNLI

In [2]:
import pandas as pd

In [3]:
df =pd.read_pickle("predictions/RL_SNLI_DEV.pickle")
df

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases
0,2,0.500000,0.0,0.166667,0.474342,0.061204,4.139196,0.535319,0.555556,2,0,0.725000,0,0,0.500000,neutral
1,2,0.333333,0.0,1.000000,0.894427,0.211949,4.704408,1.000000,1.000000,0,0,0.891667,0,0,0.666667,entailment
2,2,0.666667,0.0,0.000000,0.200000,0.063527,3.865820,0.289852,0.666667,2,0,0.516667,0,0,0.333333,contradiction
3,2,0.500000,0.0,0.750000,0.374634,0.145943,4.388149,0.888686,0.833333,1,0,0.400327,0,0,0.500000,entailment
4,2,0.500000,0.0,0.500000,0.307794,0.089295,4.352467,0.795602,0.800000,1,0,0.252941,0,0,0.500000,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9837,2,0.500000,0.0,0.500000,0.408248,0.114567,4.300111,0.583559,1.000000,1,0,0.319444,0,0,0.500000,entailment
9838,2,0.750000,0.0,0.333333,0.316228,0.070569,4.132585,0.404826,0.500000,1,0,0.602778,0,0,0.250000,contradiction
9839,2,0.333333,0.0,0.500000,0.264906,0.104411,4.334762,0.771646,1.000000,1,0,0.423977,0,0,0.666667,entailment
9840,2,0.666667,0.0,0.333333,0.229416,0.000653,4.359539,0.813604,0.500000,0,0,0.535088,0,0,0.333333,contradiction


In [4]:
original =pd.read_csv("../RIT_ollama/SNLI/DEV.csv")
original

,Unnamed: 0,gold_label,sentence1,sentence2,SET
0,549367,neutral,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,DEV
1,549368,entailment,Two women are embracing while holding to go pa...,Two woman are holding packages.,DEV
2,549369,contradiction,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,DEV
3,549370,entailment,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.,DEV
4,549371,neutral,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.,DEV
...,...,...,...,...,...
9837,559204,entailment,A small girl wearing a pink jacket is riding o...,The carousel is moving.,DEV
9838,559205,contradiction,A small girl wearing a pink jacket is riding o...,The girl is moving at the speed of light.,DEV
9839,559206,entailment,A young girl with blue and pink ribbons in her...,People in a water fountain,DEV
9840,559207,contradiction,A young girl with blue and pink ribbons in her...,A young girl knits a sweater,DEV


In [5]:
df["sentence_A"]=original["sentence1"]
df["sentence_B"]=original["sentence2"]

In [6]:
entailment=df[df["clases"]=="entailment"]
neutral=df[df["clases"]=="neutral"]
contradiction=df[df["clases"]=="contradiction"]
contradiction

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
2,2,0.666667,0.00,0.000000,0.200000,0.063527,3.865820,0.289852,0.666667,2,0,0.516667,0,0,0.333333,contradiction,Two women are embracing while holding to go pa...,The men are fighting outside a deli.
5,2,0.500000,0.00,0.000000,0.102598,0.078325,4.014813,0.558387,0.800000,3,0,0.252941,0,0,0.500000,contradiction,"Two young children in blue jerseys, one with t...",Two kids in jackets walk to school.
6,-1,1.000000,0.25,0.000000,0.000000,0.015084,3.758932,0.310552,0.000000,0,2,0.255556,0,0,0.000000,contradiction,A man selling donuts to a customer during a wo...,A woman drinks her coffee in a small cafe.
10,2,1.000000,0.00,0.000000,0.000000,0.041698,3.818552,0.234557,0.333333,1,0,0.000000,0,0,0.000000,contradiction,Two young boys of opposing teams play football...,dog eats out of bowl
14,2,0.666667,0.00,1.000000,0.288675,0.119399,4.206984,0.764824,1.000000,2,0,0.611111,0,0,0.333333,contradiction,A man in a blue shirt standing in front of a g...,A man is wearing a black shirt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9828,2,0.800000,0.00,0.200000,0.288675,0.038130,4.005370,0.270224,0.400000,1,0,0.527778,0,0,0.200000,contradiction,A man is performing an aerial skateboard trick...,A man is ordering a coffee from a barista at a...
9832,2,0.750000,0.00,0.500000,0.353553,0.021571,4.176857,0.647871,0.800000,2,0,0.583333,0,0,0.250000,contradiction,A man is performing an aerial skateboard trick...,He does a sick bicycle trick off the ramp
9834,2,0.750000,0.00,0.000000,0.239046,0.116538,4.120230,0.588300,0.750000,2,0,0.257143,0,0,0.250000,contradiction,A uniformed competitor in motocross has crosse...,A man is riding a bicycle in the park.
9838,2,0.750000,0.00,0.333333,0.316228,0.070569,4.132585,0.404826,0.500000,1,0,0.602778,0,0,0.250000,contradiction,A small girl wearing a pink jacket is riding o...,The girl is moving at the speed of light.


In [7]:
df_e=entailment.sample(n=200)
df_n=neutral.sample(n=200)
df_c=contradiction.sample(n=200)
df_t=pd.concat([df_e,df_n,df_c])
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
6009,1,0.000000,0.0,0.500000,0.353553,0.254791,4.161840,0.595915,1.000000,1,0,0.416667,0,0,1.000000,entailment,The dog is walking in the snow.,The snow is outside
6013,2,0.250000,0.0,0.666667,0.670820,0.237988,4.470784,0.922773,1.000000,1,0,0.783333,0,0,0.750000,entailment,Two men walking their canoe through the woods.,a couple of men walk their canoe
8803,2,0.333333,0.0,0.333333,0.387298,0.073237,4.243051,0.735841,0.500000,0,0,0.522222,0,0,0.666667,entailment,A man in his 30's is sitting on a curb of a bu...,A costumed figure is sitting on the curb.
2970,2,0.500000,0.0,0.500000,0.174078,0.109343,4.121398,0.626901,0.666667,1,0,0.314815,0,0,0.500000,entailment,"There is a white building, with several people...",There are four vehicles present.
3688,2,0.500000,0.0,0.500000,0.436436,0.184308,4.318469,0.856524,1.000000,1,0,0.325397,0,0,0.500000,entailment,Boy reading book with company of a rolled over...,The kid is with his dog.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,2,0.500000,0.0,0.250000,0.547723,0.084415,4.210647,0.571965,0.400000,0,0,0.588889,0,0,0.500000,contradiction,Two children playing on the floor with toy tra...,The girls slept on the floor next to the toys.
9265,2,1.000000,0.0,0.333333,0.333333,0.036740,3.897424,0.336676,0.400000,1,0,0.555556,0,0,0.000000,contradiction,The brown and white dog is playing in the snow.,A dog chasing a squirrel in a dry desert.
2270,2,0.750000,0.0,0.500000,0.500000,0.094706,4.179998,0.619938,0.800000,2,0,0.788889,0,0,0.250000,contradiction,A girl in a flower dress is running on sand.,The girl is wearing a green striped dress.
1443,2,0.750000,0.0,0.333333,0.288675,0.147231,4.354230,0.669921,0.750000,1,0,0.638889,0,0,0.250000,contradiction,Three men converse in a crowd.,Three men talk while laying down


In [8]:
lista_respuestasOllama=[]

In [9]:
df.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

In [10]:
faltantes=[]

In [11]:
df_t=df_t.reset_index(drop=True)
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,1,0.000000,0.0,0.500000,0.353553,0.254791,4.161840,0.595915,1.000000,1,0,0.416667,0,0,1.000000,entailment,The dog is walking in the snow.,The snow is outside
1,2,0.250000,0.0,0.666667,0.670820,0.237988,4.470784,0.922773,1.000000,1,0,0.783333,0,0,0.750000,entailment,Two men walking their canoe through the woods.,a couple of men walk their canoe
2,2,0.333333,0.0,0.333333,0.387298,0.073237,4.243051,0.735841,0.500000,0,0,0.522222,0,0,0.666667,entailment,A man in his 30's is sitting on a curb of a bu...,A costumed figure is sitting on the curb.
3,2,0.500000,0.0,0.500000,0.174078,0.109343,4.121398,0.626901,0.666667,1,0,0.314815,0,0,0.500000,entailment,"There is a white building, with several people...",There are four vehicles present.
4,2,0.500000,0.0,0.500000,0.436436,0.184308,4.318469,0.856524,1.000000,1,0,0.325397,0,0,0.500000,entailment,Boy reading book with company of a rolled over...,The kid is with his dog.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.500000,0.0,0.250000,0.547723,0.084415,4.210647,0.571965,0.400000,0,0,0.588889,0,0,0.500000,contradiction,Two children playing on the floor with toy tra...,The girls slept on the floor next to the toys.
596,2,1.000000,0.0,0.333333,0.333333,0.036740,3.897424,0.336676,0.400000,1,0,0.555556,0,0,0.000000,contradiction,The brown and white dog is playing in the snow.,A dog chasing a squirrel in a dry desert.
597,2,0.750000,0.0,0.500000,0.500000,0.094706,4.179998,0.619938,0.800000,2,0,0.788889,0,0,0.250000,contradiction,A girl in a flower dress is running on sand.,The girl is wearing a green striped dress.
598,2,0.750000,0.0,0.333333,0.288675,0.147231,4.354230,0.669921,0.750000,1,0,0.638889,0,0,0.250000,contradiction,Three men converse in a crowd.,Three men talk while laying down


In [12]:
df_t.to_pickle("predictions/RL_SNLI_DEV_600.pickle")

# RIT base

In [13]:
for i in range(1):
    for index,strings in df_t[i*500:(i+1)*500].iterrows():
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_base_SNLI_DEV_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*500,(i+1)*500)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
for i in range(1,2):
    for index,strings in df_t[i*500:(i+1)*500].iterrows():
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_base_SNLI_DEV_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*500,(i+1)*500)

500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
Primer rango:  500 1000


In [15]:
faltantes

[]

In [17]:
print(len(lista_respuestasOllama))

600
